# **Prediction Model_PFRHSC** 

### Importing necessary libraries

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from keras.regularizers import l2
from tensorflow.keras.models import load_model

### Mounting our dataset

In [2]:
data = pd.read_csv('Test data.csv')
test_df = pd.DataFrame(data)
test_df.describe()

,Fiber,Temperature,Compressive Strength
count,20.000000,20.000000,20.000000
mean,0.750000,365.000000,39.791000
std,0.573539,269.307103,8.809767
min,0.000000,25.000000,22.570000
25%,0.375000,150.000000,35.587500
50%,0.750000,350.000000,40.145000
75%,1.125000,550.000000,45.207500
max,1.500000,750.000000,53.660000


In [3]:
X = test_df[['Fiber', 'Temperature']]
y = test_df[['Compressive Strength']]
print(X.shape)
print(y.shape)

(20, 2)
(20, 1)


In [4]:
y

,Compressive Strength
0,50.05
1,39.23
2,36.96
3,34.08
4,22.57
5,50.79
6,43.55
7,39.40
8,36.09
9,25.26


### Splitting the dataset

In [5]:
X_train, X_, y_train, y_ = train_test_split(X, y, test_size = 0.4, random_state = 1)
X_cv, X_test, y_cv, y_test = train_test_split(X_, y_, test_size = 0.5, random_state = 1)

In [6]:
print(X_train.shape)
print(X_cv.shape)
print(X_test.shape)
print(y_train.shape)

(12, 2)
(4, 2)
(4, 2)
(12, 1)


### Preparing the data for cross-validation

In [7]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_cv_scaled = scaler.transform(X_cv)
X_test_scaled = scaler.transform(X_test)

### Building different architecture for our model

In [8]:
def ANN_model(reg):
    tf.random.set_seed(20)
    
    model_1 = Sequential(
        [
            Dense(2, activation='relu', kernel_regularizer=l2(reg)),
            Dense(4, activation='relu', kernel_regularizer=l2(reg)),
            Dense(1, activation='linear')
        ],
        name='model_1'
    )

    model_2 = Sequential(
        [
            Dense(4, activation='relu', kernel_regularizer=l2(reg)),
            Dense(4, activation='relu', kernel_regularizer=l2(reg)),
            Dense(2, activation='relu', kernel_regularizer=l2(reg)),
            Dense(1, activation='linear'),
        ],
        name='model_2'
    )

    model_3 = Sequential(
        [
            Dense(4, activation='relu', kernel_regularizer=l2(reg)),
            Dense(3, activation='relu', kernel_regularizer=l2(reg)),
            Dense(1, activation='linear'),
        ],
        name='model_3'
    )
    
    model_4 = Sequential(
        [
            Dense(5, activation='relu', kernel_regularizer=l2(reg)),
            Dense(3, activation='relu', kernel_regularizer=l2(reg)),
            Dense(1, activation='linear'),
        ],
        name='model_4'
    )
    
    model_5 = Sequential(
        [
            Dense(3, activation='relu', kernel_regularizer=l2(reg)),
            Dense(5, activation='relu', kernel_regularizer=l2(reg)),
            Dense(2, activation='relu', kernel_regularizer=l2(reg)),
            Dense(1, activation='linear')
        ],
        name='model_5'
    )
    
    model_6 = Sequential(
        [
            Dense(4, activation='relu', kernel_regularizer=l2(reg)),
            Dense(5, activation='relu', kernel_regularizer=l2(reg)),
            Dense(2, activation='relu', kernel_regularizer=l2(reg)),
            Dense(1, activation='linear')
        ],
        name='model_6'
    )   
    
    model_list = [model_1, model_2, model_3, model_4, model_5, model_6]
    
    return model_list
    

In [9]:
def train_and_evaluate_model(model, X_train, y_train, X_cv, y_cv, model_identity):
    model.compile(
        loss='mean_squared_error',
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.01)
    )

    print(f"Model's name: {model.name}..." + 
         f"With Regularization: {reg}")
    model.fit(
        X_train, y_train,
        epochs=10000,
        verbose=0
    )
    
    model.save(f"{model_name}.keras")
    
    yhat_train = model.predict(X_train)
    train_mse = mean_squared_error(y_train, yhat_train) / 2
    print(f"Training data MSE: {train_mse}")

    yhat_cv = model.predict(X_cv)
    cv_mse = mean_squared_error(y_cv, yhat_cv) / 2
    print(f"CV data MSE: {cv_mse}")

    r2 = r2_score(y_cv, yhat_cv)
    print(f"R2 value: {r2}\n\n")

regularization_strengths = [0, 0.01, 0.001, 0.0001]
saved_models = {}

for reg in regularization_strengths:
    models_with_reg = ANN_model(reg)

    
    for i, model in enumerate(models_with_reg):
        model_name = f"model_{i + 1}_reg_{reg}"
        train_and_evaluate_model(model, X_train_scaled, y_train, X_cv_scaled, y_cv, model_name)
        
        saved_models[model_name] = load_model(f"{model_name}.keras")


Model's name: model_1...With Regularization: 0
1/1 [==============================] - 0s 58ms/step
Training data MSE: 2.5079005126829577
1/1 [==============================] - 0s 20ms/step
CV data MSE: 2.9792855764964004
R2 value: 0.9192091765945292


Model's name: model_2...With Regularization: 0
1/1 [==============================] - 0s 35ms/step
Training data MSE: 2.506913008171666
1/1 [==============================] - 0s 14ms/step
CV data MSE: 2.964228796354578
R2 value: 0.9196174790664672


Model's name: model_3...With Regularization: 0
1/1 [==============================] - 0s 34ms/step
Training data MSE: 32.60215279438362
1/1 [==============================] - 0s 14ms/step
CV data MSE: 42.22511365953153
R2 value: -0.1450401827390142


Model's name: model_4...With Regularization: 0
1/1 [==============================] - 0s 42ms/step
Training data MSE: 3.3159431634668644
1/1 [==============================] - 0s 14ms/step
CV data MSE: 2.581126009962006
R2 value: 0.930006274892473

In [10]:
saved_models

{'model_1_reg_0': <keras.src.engine.sequential.Sequential at 0x1d17e9ecf50>,
 'model_2_reg_0': <keras.src.engine.sequential.Sequential at 0x1d1000ece90>,
 'model_3_reg_0': <keras.src.engine.sequential.Sequential at 0x1d17d8dff50>,
 'model_4_reg_0': <keras.src.engine.sequential.Sequential at 0x1d102242f90>,
 'model_5_reg_0': <keras.src.engine.sequential.Sequential at 0x1d103a24110>,
 'model_6_reg_0': <keras.src.engine.sequential.Sequential at 0x1d103e3a650>,
 'model_1_reg_0.01': <keras.src.engine.sequential.Sequential at 0x1d10532fc10>,
 'model_2_reg_0.01': <keras.src.engine.sequential.Sequential at 0x1d1069fca10>,
 'model_3_reg_0.01': <keras.src.engine.sequential.Sequential at 0x1d106805890>,
 'model_4_reg_0.01': <keras.src.engine.sequential.Sequential at 0x1d105327c10>,
 'model_5_reg_0.01': <keras.src.engine.sequential.Sequential at 0x1d1089e5f50>,
 'model_6_reg_0.01': <keras.src.engine.sequential.Sequential at 0x1d109f552d0>,
 'model_1_reg_0.001': <keras.src.engine.sequential.Sequent

In [11]:
y_test

,Compressive Strength
17,42.86
6,43.55
16,46.43
4,22.57


### Based on the trial results, "model_2_reg_0.001" is selected

In [30]:
selected_model = saved_models['model_2_reg_0.001']

print(f"Selected Model: {selected_model.name}")
yhat_test = selected_model.predict(X_test_scaled)
test_mse = mean_squared_error(y_test, yhat_test) / 2
print(f"Mean Squared Error (MSE) for test data:{test_mse}")
test_mae = mean_absolute_error(y_test, yhat_test)
print(f"Mean Absolute Error (MAE) for test data:{test_mae}")
test_r2 = r2_score(y_test, yhat_test)
print(f"R2 for test data:{test_r2}")
test_rmse = np.sqrt(test_mse)
print(f"Root Mean Squared Error (RMSE) for test data:{test_rmse}")

Selected Model: model_2
1/1 [==============================] - 0s 18ms/step
Mean Squared Error (MSE) for test data:0.81966696486235
Mean Absolute Error (MAE) for test data:1.2617386436462397
R2 for test data:0.9818187570417031
Root Mean Squared Error (RMSE) for test data:0.9053546072464369


In [22]:
print(f"Original: \n {y_test['Compressive Strength']}" + 
      f"\nPrediction: \n{yhat_test}")

Original: 
 17    42.86
6     43.55
16    46.43
4     22.57
Name: Compressive Strength, dtype: float64
Prediction: 
[[44.21087 ]
 [41.98584 ]
 [47.57763 ]
 [23.554296]]


### Combining all the data back again for the final prediction

In [14]:
combined_X = np.vstack([X_train_scaled, X_cv_scaled, X_test_scaled])
combined_y = np.concatenate([y_train, y_cv, y_test])

In [23]:
selected_model = saved_models[ 'model_2_reg_0.001']

print(f"Selected Model: {selected_model.name}")
yhat = selected_model.predict(combined_X)
mse = mean_squared_error(combined_y, yhat) / 2
print(f"Mean Squared Error (MSE):{mse}")
mae = mean_absolute_error(combined_y, yhat)
print(f"Mean Absolute Error (MAE):{mae}")
r2 = r2_score(combined_y, yhat)
print(f"R2:{r2}")
rmse = np.sqrt(mse)
print(f"Root Mean Squared Error (RMSE):{rmse}")

Selected Model: model_2
1/1 [==============================] - 0s 10ms/step
Mean Squared Error (MSE):0.20063427440771558
Mean Absolute Error (MAE):0.39243529510498
R2:0.9945576971455748
Root Mean Squared Error (RMSE):0.4479221744987801


### Final outputs

In [24]:
original_series = pd.Series(combined_y.flatten(), name="Original")
prediction_series = pd.Series(yhat.flatten(), name="Prediction")

df_ANN = pd.concat([original_series, prediction_series], axis=1)

df_ANN

,Original,Prediction
0,39.40,39.378170
1,39.23,39.243782
2,39.35,38.744503
3,50.05,50.119289
4,28.78,28.701527
5,40.89,41.184216
6,25.26,25.270035
7,53.66,53.629589
8,36.09,36.304787
9,41.82,41.794525


In [25]:
df_ANN.to_csv('ANN_original_VS_predictions.csv', index=False)

In [29]:
model_weights = selected_model.get_weights()

for i, layer_weights in enumerate(model_weights):
    print(f"Layer {i + 1} Weights:")
    print(layer_weights)

Layer 1 Weights:
[[ 4.1725002e-02 -1.9175401e-02  2.4780864e-01 -6.3430578e-02]
 [-1.7188708e+00  1.1796446e-03  1.0777767e+00  1.5895038e+00]]
Layer 2 Weights:
[ 2.0496478 -0.5949067  1.1972617  1.1166148]
Layer 3 Weights:
[[ 2.3330832e+00  7.4186665e-01 -7.2785586e-01  9.8711002e-35]
 [ 2.4571412e-03  1.2887592e-02  3.9651394e-03 -5.5243275e-34]
 [ 1.3997397e+00  7.5901443e-01  4.5955694e-01  5.7913030e-34]
 [ 1.1112046e+00  3.1871349e-01  1.3766140e+00 -4.5570298e-34]]
Layer 4 Weights:
[ 0.43030366 -1.9753298  -3.410853   -0.07102266]
Layer 5 Weights:
[[ 1.6266965e+00 -3.3374913e-08]
 [ 1.7398837e+00  2.6121989e-03]
 [-2.4229751e+00 -1.0127704e-02]
 [-4.8566293e-34 -2.4522059e-34]]
Layer 6 Weights:
[ 0.6326311  -0.18578708]
Layer 7 Weights:
[[ 2.6281366]
 [-1.1858653]]
Layer 8 Weights:
[0.6461619]
